# Telecom_DC

Клетка для необходимых включений

In [128]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold,RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.linear_model import SGDClassifier
import numpy as np


import xgboost as xgb
def list_to_float(data,list_):
    temp_data = data
    for el in list_:
        temp_data[el] = to_float(temp_data,el)
    return temp_data

# Вспомогательная функция
def to_float(data,col):
    return data[col].apply(lambda x: float('0.'+str(x[1:])))


def calculate_means(numeric_data):
    means = np.zeros(numeric_data.shape[1])
    for j in range(numeric_data.shape[1]):
        to_sum = numeric_data.iloc[:,j]
        indices = np.nonzero(~numeric_data.iloc[:,j].isnull())[0]
        correction = np.amax(to_sum[indices])
        to_sum /= correction
        for i in indices:
            means[j] += to_sum[i]
        means[j] /= indices.size
        means[j] *= correction
    return pd.Series(means, numeric_data.columns)


import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

def plot_roc_auc(y_test,pred):
    fpr, tpr, _ = roc_curve(y_test, pred)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
     lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

def transoform(row):
    if row['SUM_DATA_MIN'] != 0:
        return row['SUM_DATA_MB']/row['SUM_DATA_MIN']
    else:
        return 0

In [129]:
def test_xgboost(data):

    param = {'objective':'binary:logistic', 'n_estimators':20}

    model_boost = xgb.XGBModel(**param)
# Or you can use: clf = xgb.XGBClassifier(**param_dist)
    X = data.drop(['CSI','SK_ID'],axis=1)
    y = data['CSI']
    corr = []
 #   print(X)
    skf = StratifiedKFold(n_splits=4,shuffle=True,random_state=0)
    ra_boost = []
    counter = 0
    param_grid = {'n_estimaros':[100,40,20]}
    for train_index, test_index in skf.split(X, y):

     
        
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = y[train_index], y[test_index]
#        rocauc = roc_auc_score(pred,y_test)
#        print(rocauc)
        model_boost.fit(X_train,y_train,eval_metric='auc',verbose=True)
        pred = model_boost.predict(X_test)
        ra_boost.append(roc_auc_score(y_test,pred))
        predictions = model_boost.predict(X_site)
#        print('{} - TEST, {} - correlation'.format(roc_auc_score(y_test,pred),roc_auc_score(best,predictions)))
#    print('{} - correlation'.format(roc_auc_score(best,predictions)))
    corr.append(roc_auc_score(best,predictions))
    #print(ra_boost)
#    print('{} - average rocauc'.format(sum(ra_boost, 0.0) / len(ra_boost)))
#    plot_roc_auc(y_test,model_boost.predict(X_test))
#    print(model_boost.feature_importances_)
    print(sum(corr, 0.0)/ len(corr))
    return model_boost

## Подключение основной таблицы csi_train

In [130]:
data_base = pd.read_csv('subs_csi_train.csv',sep=';')
data_base = data_base.drop(['CONTACT_DATE'],axis=1)
data_base.head()

,SK_ID,CSI
0,2,0
1,3,0
2,4,0
3,5,0
4,6,1


Данных явно недостаточно для того чтобы сделать хоть какое-нибудь предсказание. Расммотрим следующую таблицу

##  Подключение таблицы subs_bs_voice_session

In [131]:
data_voice_session = pd.read_csv('subs_bs_voice_session_train.csv',sep=';')
data_voice_session['VOICE_DUR_MIN']=data_voice_session['VOICE_DUR_MIN'].apply(lambda x: float('0.'+str(x[1:])))
data_voice_session = data_voice_session.groupby(['CELL_LAC_ID','SK_ID']).agg({'VOICE_DUR_MIN': np.sum})
data_voice_session = data_voice_session.reset_index(level=['CELL_LAC_ID', 'SK_ID'])
data_voice_session.head()

,CELL_LAC_ID,SK_ID,VOICE_DUR_MIN
0,1,3564,0.023958
1,2,919,0.053125
2,2,1903,0.003681
3,2,2111,0.007708
4,4,1837,0.160069


## Подключение таблицы consumptions

In [133]:
data_cons = pd.read_csv('subs_bs_consumption_train.csv',sep=';')
data_cons= list_to_float(data_cons, ['SUM_MINUTES','SUM_DATA_MB','SUM_DATA_MIN'])
data_cons = data_cons.groupby(['CELL_LAC_ID','SK_ID']).agg({'SUM_MINUTES':np.sum,
                                                 'SUM_DATA_MB':np.sum,
                                                 'SUM_DATA_MIN':np.sum})
data_cons = data_cons.fillna(0)
data_cons = data_cons.reset_index(level=['CELL_LAC_ID', 'SK_ID'])
data_cons['SPEED'] = data_cons.apply(lambda row: transoform(row),axis=1)


data_cons = data_cons.groupby(['SK_ID']).agg({'SUM_MINUTES':np.sum,
                                                 'SUM_DATA_MB':np.sum,
                                                 'SUM_DATA_MIN':np.sum,'SPEED':np.mean})
data_cons = data_cons.reset_index(level=['SK_ID'])
data_cons[20:40]

,SK_ID,SUM_MINUTES,SUM_DATA_MB,SUM_DATA_MIN,SPEED
20,22,0.387132,0.011387,1.095266,0.001447
21,23,0.053921,0.000004,0.000046,0.007919
22,24,0.131351,0.079960,2.980037,0.026343
23,25,0.193019,0.086934,2.521596,0.108965
24,26,0.091638,0.000000,0.000000,0.000000
25,27,0.003593,0.000000,0.000000,0.000000
26,28,0.202327,0.090305,1.842172,0.045247
27,30,0.228111,0.175517,2.105845,0.021082
28,31,0.128297,0.000000,0.000000,0.000000
29,33,0.119104,0.000000,0.000000,0.000000


## Поключение sub_data_session

In [135]:
data_data_session = pd.read_csv('subs_bs_data_session_train.csv',sep=';')
data_data_session['DATA_VOL_MB'] = to_float(data_data_session, 'DATA_VOL_MB')
data_data_session = data_data_session.groupby(['CELL_LAC_ID','SK_ID']).agg({'DATA_VOL_MB': np.sum})
data_data_session = data_data_session.reset_index(level=['CELL_LAC_ID', 'SK_ID'])
data_data_session = data_data_session.groupby('SK_ID').agg({'DATA_VOL_MB':np.mean})
data_data_session = data_data_session.reset_index(level=['SK_ID'])

data_data_session.head()


,SK_ID,DATA_VOL_MB
0,2,0.000731
1,3,0.004615
2,5,0.015402
3,7,0.003424
4,9,0.000009


## Подключение sub_features

In [137]:
data_sub_feat = pd.read_csv('subs_features_train.csv',sep=';')

data_sub_feat = data_sub_feat.drop(['COM_CAT#24'],axis=1)

list_float = ['COM_CAT#'+str(i) for i in [19,20,21,22,23,31,32,33,27,28,29,30,18,17]]
list_float.extend(['COST','REVENUE','ITC','VAS','RENT_CHANNEL','ROAM'])

data_sub_feat = list_to_float(data_sub_feat, list_float)

data_sub_feat = data_sub_feat.groupby(['SNAP_DATE','SK_ID']).agg({'COM_CAT#20':np.sum,
                                                 'COM_CAT#22':np.sum,
                                                 'COM_CAT#23':np.sum,
                   
                                                 'COM_CAT#31':np.sum,
                                                 
                                                 'COM_CAT#33':np.sum,
                                            'COM_CAT#1':np.sum,
                                          'COM_CAT#7':np.sum,                            
                                          'COM_CAT#8':np.sum,
                                         'COM_CAT#20':np.sum,
                                            'ROAM':np.sum,
                                        'INTERNET_TYPE_ID':np.mean})
data_sub_feat = data_sub_feat.reset_index(level=['SK_ID','SNAP_DATE'])
data_sub_feat = data_sub_feat[data_sub_feat['SNAP_DATE'] == max(data_sub_feat['SNAP_DATE'])]
data_sub_feat = data_sub_feat.drop(['SNAP_DATE'],axis=1)
data_sub_feat


,SK_ID,INTERNET_TYPE_ID,COM_CAT#31,COM_CAT#1,COM_CAT#8,COM_CAT#33,ROAM,COM_CAT#23,COM_CAT#20,COM_CAT#7,COM_CAT#22
43443,3,2.0,0.026669,4,742.0,0.103896,0.000000,0.123837,0.702392,2,0.699855
43444,4,3.0,0.033402,8,1268.0,0.103896,0.000000,0.114931,0.701234,3,0.701234
43445,5,2.0,0.051575,3,110.0,0.103896,0.000000,0.129319,0.705282,1,0.705282
43446,6,3.0,0.010953,5,937.0,0.103896,0.000000,0.109409,0.698438,4,0.698438
43447,7,1.0,0.078902,7,1133.0,0.103896,0.000000,0.127944,0.704130,2,0.704130
43448,8,3.0,0.024180,5,1244.0,0.103896,0.000000,0.122953,0.700102,4,0.700102
43449,9,2.0,0.002475,5,1325.0,0.103896,0.000000,0.110035,0.698818,3,0.698818
43450,10,3.0,0.073965,1,1119.0,0.103896,0.000000,0.131422,0.705667,7,0.705667
43451,11,1.0,0.000000,8,142.0,0.103896,0.000000,0.109409,0.698417,6,0.698417
43452,12,2.0,0.068480,2,126.0,0.104547,0.000000,0.135986,0.707473,1,0.707473


## Пользовался ли юзер "плохой" клеткой?

In [138]:
data = pd.merge(data_base,data_cons,on='SK_ID',how='left')
unique, counts = np.unique(data[data['CSI']==1].values[:,2],return_counts=True)
a = dict(zip(unique, counts))
new_dict = {'CELL_LAC_ID':list(a.keys()),'NUM':list(a.values())}
data_cell = pd.DataFrame.from_dict(new_dict)
data = pd.merge(data,data_cell, on='CELL_LAC_ID',how='left')
data = data.fillna(0)
data = data.groupby(['SK_ID']).agg({'NUM':np.mean,
                                                 'SPEED':np.mean,
                                                 'SUM_DATA_MIN':np.sum})
data = data.reset_index(level=['SK_ID'])
data = pd.merge(data,data_base,on='SK_ID',how='left')
data

KeyError: 'CELL_LAC_ID'

## Обединение таблицы

In [ ]:
gen_dat_cons = pd.merge(data_base,data_cons,on='SK_ID',how='left')
gen_dat_cons
#gen_dat_cons = gen_dat_cons.fillna(calculate_means(gen_dat_cons))

gen_dat = pd.merge(gen_dat_cons,data_data_session,on='SK_ID',how='left')
X = gen_dat.drop(['SK_ID','CSI'],axis=1).fillna(0)
y = gen_dat['CSI']

scaler = StandardScaler()
X = scaler.fit_transform(X)
gen_dat = gen_dat.fillna(0)
X

#gen_dat_voice = pd.merge(data_base,data_voice_session,on='SK_ID',how='left')
#gen_dat_voice = gen_dat_voice.fillna(calculate_means(gen_dat_voice))
#gen_dat_voice

#gen_dat_dat = pd.merge(data_base,data_data_session, on='SK_ID',how='left')
#gen_dat_dat = gen_dat_dat.fillna(calculate_means(gen_dat_dat))
#gen_dat_dat

#gen_dat_feat = pd.merge(data_base, data_sub_feat,on='SK_ID',how='left')
#gen_dat_feat = gen_dat_feat.fillna(calculate_means(gen_dat_feat))

#gen_dat_feat
#X = gen_dat_feat.drop(['CSI','SK_ID'],axis=1)
#y = gen_dat_feat['CSI']


# Построение модели

###  XGBoost

In [127]:
gen_m = test_xgboost(gen_dat)
gen_m.feature_importances_

0.6941471557862168


array([0.32352942, 0.15441176, 0.23529412, 0.09558824, 0.19117647],
      dtype=float32)

In [99]:
model = RandomForestClassifier()
#pred1 = cons_m.predict(gen_dat_cons.drop(['CSI','SK_ID'],axis=1))
gen_dat_cons

,SK_ID,CSI,CELL_LAC_ID,SUM_MINUTES,SUM_DATA_MIN,SUM_DATA_MB,SPEED
0,0.000425,0.0,0.002621,0.000000,0.000090,5.202471e-06,0.000967
1,0.000425,0.0,0.009506,0.000000,0.000090,1.274075e-06,0.000237
2,0.000425,0.0,0.010456,0.000063,0.000090,3.609878e-06,0.000670
3,0.000425,0.0,0.013166,0.000022,0.000000,0.000000e+00,0.000000
4,0.000425,0.0,0.030858,0.000004,0.001048,1.238139e-05,0.000198
5,0.000425,0.0,0.047568,0.000000,0.000090,2.858501e-07,0.000053
6,0.000425,0.0,0.048890,0.000015,0.000000,0.000000e+00,0.000000
7,0.000425,0.0,0.050712,0.000042,0.000000,0.000000e+00,0.000000
8,0.000425,0.0,0.051336,0.000075,0.000000,0.000000e+00,0.000000
9,0.000425,0.0,0.066649,0.000033,0.000006,9.906746e-06,0.029613


In [12]:
from deslib.dcs.a_priori import APriori
from deslib.dcs.mcb import MCB
from deslib.dcs.ola import OLA
from deslib.des.des_p import DESP
from deslib.des.knora_e import KNORAE
from deslib.des.knora_u import KNORAU
from deslib.des.meta_des import METADES
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,
                                                    random_state=0)

# Scale the variables to have 0 mean and unit variance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Split the data into training and DSEL for DS techniques
X_train, X_dsel, y_train, y_dsel = train_test_split(X_train, y_train,
                                                    test_size=0.5,
                                                    random_state=0)

pool_classifiers = RandomForestClassifier(n_estimators=100, random_state=0)
pool_classifiers.fit(X_train, y_train)

# Initialize the DS techniques
knorau = KNORAU(pool_classifiers)
kne = KNORAE(pool_classifiers)
desp = DESP(pool_classifiers)
ola = OLA(pool_classifiers)
mcb = MCB(pool_classifiers)

calibrated_pool = []
for clf in pool_classifiers:
    calibrated = CalibratedClassifierCV(base_estimator=clf, cv='prefit')
    calibrated.fit(X_dsel, y_dsel)
    calibrated_pool.append(calibrated)

apriori = APriori(calibrated_pool)
meta = METADES(calibrated_pool)


knorau.fit(X_dsel, y_dsel,)
kne.fit(X_dsel, y_dsel)
desp.fit(X_dsel, y_dsel)
ola.fit(X_dsel, y_dsel)
mcb.fit(X_dsel, y_dsel)
apriori.fit(X_dsel, y_dsel)


In [16]:
print('Evaluating DS techniques:')
print('Classification accuracy KNORA-Union: ',
      knorau.score(X_test, y_test))
print('Classification accuracy KNORA-Eliminate: ',
      kne.score(X_test, y_test))
print('Classification accuracy DESP: ', roc_auc_score(desp.predict(X_test), y_test))
print('Classification accuracy OLA: ', ola.score(X_test, y_test))
print('Classification accuracy A priori: ', apriori.score(X_test, y_test))
print('Classification accuracy MCB: ', mcb.score(X_test, y_test))
print('Classification accuracy Bagging: ',
      pool_classifiers.score(X_test, y_test))

Evaluating DS techniques:
Classification accuracy KNORA-Union:  0.8228120516499282
Classification accuracy KNORA-Eliminate:  0.7840746054519369
Classification accuracy DESP:  0.4807894238548534
Classification accuracy OLA:  0.7682926829268293
Classification accuracy A priori:  0.8464849354375896
Classification accuracy MCB:  0.7309899569583931
Classification accuracy Bagging:  0.8185078909612625


In [74]:
desp.predict(X_test)

array([0., 0., 0., ..., 0., 0., 0.])

In [75]:
X_test.shape

(1394, 10)

# Подготовка решениия

In [23]:
test_data_base = pd.read_csv('subs_csi_test.csv',sep=';')
test_data_base = test_data_base.drop(['CONTACT_DATE'],axis=1)
test_data_base.head()

test_data_cons = pd.read_csv('subs_bs_consumption_test.csv',sep=';')
test_data_cons= list_to_float(test_data_cons, ['SUM_MINUTES','SUM_DATA_MB','SUM_DATA_MIN'])
test_data_cons = test_data_cons.groupby(['CELL_LAC_ID','SK_ID']).agg({'SUM_MINUTES':np.sum,
                                                 'SUM_DATA_MB':np.sum,
                                                 'SUM_DATA_MIN':np.sum})
test_data_cons = test_data_cons.fillna(0)
test_data_cons = test_data_cons.reset_index(level=['CELL_LAC_ID', 'SK_ID'])
test_data_cons['SPEED'] = test_data_cons.apply(lambda row: transoform(row),axis=1)
test_data_cons = test_data_cons.groupby(['SK_ID']).agg({'SUM_MINUTES':np.sum,
                                                 'SUM_DATA_MB':np.sum,
                                                 'SUM_DATA_MIN':np.sum,'SPEED':np.mean})

test_data = pd.merge(test_data_base,test_data_cons,on='SK_ID',how='left')

'''
test_data = pd.merge(test_data,data_cell, on='CELL_LAC_ID',how='left')
test_data = test_data.fillna(-100)
test_data = test_data.groupby(['SK_ID']).agg({'NUM':np.mean,
                                                 'SPEED':np.mean,
                                                 'SUM_DATA_MIN':np.sum})
test_data = test_data.reset_index(level=['SK_ID'])
test_data = pd.merge(test_data,test_data_base,on='SK_ID',how='left')


test_data_sub_feat = pd.read_csv('subs_features_test.csv',sep=';')

test_data_sub_feat = test_data_sub_feat.drop(['COM_CAT#24'],axis=1)

list_float = ['COM_CAT#'+str(i) for i in [19,20,21,22,23,31,32,33,27,28,29,30,18,17]]
list_float.extend(['COST','REVENUE','ITC','VAS','RENT_CHANNEL','ROAM'])

test_data_sub_feat = list_to_float(test_data_sub_feat, list_float)

test_data_sub_feat = test_data_sub_feat.groupby(['SNAP_DATE','SK_ID']).agg({'COM_CAT#20':np.sum,
                                                 'COM_CAT#22':np.sum,
                                                 'COM_CAT#23':np.sum,
                                                 
                                                                                           
                                                 'COM_CAT#31':np.sum,
                                                 
                                                 'COM_CAT#33':np.sum,
                                          
                                          'COM_CAT#1':np.sum,
                                       
                                          'COM_CAT#7':np.sum,
                             
                                          'COM_CAT#8':np.sum,
                                         'COM_CAT#20':np.sum,
                                            'ROAM':np.sum,
                                          
                                          'INTERNET_TYPE_ID':np.mean})
                                                
test_data_sub_feat = test_data_sub_feat.reset_index(level=['SK_ID','SNAP_DATE'])
test_data_sub_feat = test_data_sub_feat[test_data_sub_feat['SNAP_DATE'] == max(test_data_sub_feat['SNAP_DATE'])]
test_data_sub_feat = test_data_sub_feat.drop(['SNAP_DATE'],axis=1)

gen_dat_feat = pd.merge(data_base, data_sub_feat,on='SK_ID',how='left')
gen_dat_feat = gen_dat_feat.fillna(calculate_means(gen_dat_feat))
'''

test_data_session = pd.read_csv('subs_bs_data_session_test.csv',sep=';')
test_data_session['DATA_VOL_MB'] = to_float(test_data_session, 'DATA_VOL_MB')
test_data_session = test_data_session.groupby(['CELL_LAC_ID','SK_ID']).agg({'DATA_VOL_MB': np.sum})
test_data_session = test_data_session.reset_index(level=['CELL_LAC_ID', 'SK_ID'])
test_data_session = test_data_session.groupby('SK_ID').agg({'DATA_VOL_MB':np.mean})
test_data_session = test_data_session.reset_index(level=['SK_ID'])



test_data = pd.merge(test_data,test_data_session,on='SK_ID',how='left')

X_site = test_data.drop(['SK_ID'],axis=1).fillna(0).values
test_data


,SK_ID,SUM_MINUTES,SUM_DATA_MB,SUM_DATA_MIN,SPEED,DATA_VOL_MB
0,6184,0.228911,8.928420e-03,1.398838,0.007077,6.007085e-04
1,5175,0.191383,3.840382e-01,1.835840,0.079624,2.380667e-02
2,5478,0.263264,0.000000e+00,0.000000,0.000000,NaN
3,7807,0.424628,6.433734e-02,0.575891,0.035065,1.837513e-02
4,5640,0.473312,3.270672e-01,1.892111,0.074019,8.535722e-02
5,6083,0.012402,0.000000e+00,0.000000,0.000000,NaN
6,2707,0.515380,2.499396e-01,2.351601,0.017065,2.109862e-01
7,6681,0.129844,7.573060e-03,0.018886,0.044487,7.017378e-03
8,8976,0.344459,1.945571e-02,0.068417,0.101434,9.039729e-03
9,9526,0.178368,8.222160e-03,1.068129,0.046656,9.369539e-04


In [91]:
#X_test_site = test_data.drop(['SK_ID','NUM'],axis=1).fillna(0)
predictions = gen_m.predict(X_site)
#predictions = model_boost.predict(X_test_site.values)
with open('1.csv','w') as fout:
    for el in predictions:
        fout.write(str(el)+'\n')
best = pd.read_csv('best53.csv')
best = np.vstack((np.array(0),np.array(best)))
roc_auc_score(best,predictions)

0.7320881736542137

### Проверка с лушчим решением


In [113]:
best54 = pd.read_csv('best54.csv')
current = pd.read_csv('1.csv')
random = pd.read_csv('best53.csv')

best54 = np.vstack((np.array(0.30348447),np.array(best54)))
current = np.vstack((np.array(0.18471411),np.array(current)))
random = np.vstack((np.array(0),np.array(random)))

with open('3.csv','w') as fout:
    for i in range(len(best54)):
        temp = float(best54[i])/2+float(current[i])/2
        temp += float(random[i])/2
        fout.write(str(temp)+'\n')
new = pd.read_csv('best53.csv')
new = np.vstack((np.array(0),np.array(new)))
roc_auc_score(best,new)


1.0

In [86]:
X

array([[1.04033e-01, 1.95380e-02, 2.00000e+00, ..., 1.36000e+02,
        4.00000e+00, 7.01257e-01],
       [1.03896e-01, 5.33110e-02, 1.00000e+00, ..., 1.49000e+02,
        3.00000e+00, 7.04834e-01],
       [1.03896e-01, 6.89530e-02, 3.00000e+00, ..., 1.27700e+03,
        6.00000e+00, 6.99904e-01],
       ...,
       [1.03896e-01, 1.13040e-02, 4.00000e+00, ..., 0.00000e+00,
        5.00000e+00, 6.98539e-01],
       [1.04275e-01, 7.56940e-02, 1.00000e+00, ..., 7.95000e+02,
        1.00000e+00, 7.14844e-01],
       [1.03896e-01, 3.15000e-04, 3.00000e+00, ..., 1.32400e+03,
        4.00000e+00, 6.98463e-01]])

In [87]:
test_data

,SK_ID,COM_CAT#33,COM_CAT#31,COM_CAT#7,INTERNET_TYPE_ID,COM_CAT#23,COM_CAT#20,ROAM,COM_CAT#8,COM_CAT#1,COM_CAT#22
0,6184,0.104033,0.019538,2.0,2.0,0.118528,0.701257,0.0,136.0,4.0,0.701257
1,5175,0.103896,0.053311,1.0,2.0,0.130575,0.704834,0.0,149.0,3.0,0.704834
2,5478,0.103896,0.068953,3.0,2.0,0.112392,0.699904,0.0,1277.0,6.0,0.699904
3,7807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5640,0.103896,0.081004,1.0,2.0,0.140597,0.706550,0.0,1279.0,8.0,0.706550
5,6083,0.103896,0.000704,5.0,3.0,0.109409,0.698418,0.0,1249.0,7.0,0.698418
6,2707,0.103896,0.047827,3.0,1.0,0.134624,0.706688,0.0,1575.0,5.0,0.706688
7,6681,0.103896,0.000841,1.0,1.0,0.109409,0.698453,0.0,1645.0,8.0,0.698453
8,8976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9526,0.103896,0.077593,2.0,1.0,0.122253,0.704759,0.0,0.0,7.0,0.704759
